In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset=load_dataset("isidentical/moondream2-coyo-5M-captions")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5005590 [00:00<?, ? examples/s]

In [4]:
train_dataset=dataset["train"]

NameError: name 'dataset' is not defined

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['key', 'url', 'alt_text', 'moondream2_caption', 'sha256'],
        num_rows: 5005590
    })
})

In [23]:
!pip uninstall clip -y
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-z1awaxz9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-z1awaxz9
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_

In [24]:
from PIL import Image
from io import BytesIO
import torch
from transformers import CLIPProcessor, CLIPModel
import clip

device= "cuda" if torch.cuda.is_available() else "cpu"

#model,preprocess=clip.load("ViT-B/32", device=device)





config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [36]:
model,preprocess= clip.load("ViT-B/32",device ="cpu")

100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 53.1MiB/s]


In [3]:
import requests
from PIL import Image
from io import BytesIO
from PIL import UnidentifiedImageError
from tqdm import tqdm

error_count = 0
error_urls = []
image_embeddings = []
coutn=0;

for example in tqdm(train_dataset):

    image_url = example['url']
    try:

        coutn+=1

        if coutn == 50:
          break

        response = requests.get(image_url)
        response.raise_for_status()  # Raise an exception for non-2xx status codes

        # Check if the content type is an image
        if response.headers.get('content-type', '').startswith('image'):

            image=preprocess(Image.open(BytesIO(response.content))).unsqueeze(0)

            with torch.no_grad():
              image_features=model.encode_image(image)

        else:
            raise ValueError("Not an image file")

        # Process image embeddings or other tasks here

    except (requests.exceptions.RequestException, UnidentifiedImageError, ValueError) as e:
        error_urls.append(image_url)
        error_count += 1
        continue

print(f"Total errors: {error_count}")
print(f"Error URLs: {error_urls}")


NameError: name 'train_dataset' is not defined

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objs as go
import plotly.express as px

# Example data: replace this with your actual 512-dimensional data
data = np.random.rand(100, 512)  # Assuming you have 100 data points

# Perform t-SNE to reduce dimensions to 3
tsne = TSNE(n_components=3, random_state=42)
data_3d = tsne.fit_transform(data)

# Plot the 3D scatter plot using Plotly
fig = px.scatter_3d(
    x=data_3d[:, 0],
    y=data_3d[:, 1],
    z=data_3d[:, 2],
    title="3D Scatter Plot of t-SNE Reduced Data",
    labels={'x': 'TSNE Component 1', 'y': 'TSNE Component 2', 'z': 'TSNE Component 3'}
)

fig.show()